# NRC-09 Darks: Glow Signature

|                                |                                                          |
|:---                            |:---                                                      |
|__CAR Title__                   | NRC-09: Darks                                            |
|__APT Program #__               | 1062                                                     |
|__NGAS #__                      | 102                                                      |
|__CAR Execution Dates(s) (UT):__| TBD                                                      |
|__JIRA Ticket Links:__          | CAR [NRCCOM-13](https://jira.stsci.edu/browse/NRCCOM-13) ||
|                                | CAP [NRCCOM-14](https://jira.stsci.edu/browse/NRCCOM-14) ||
|                                | [JSOCOPS-15](https://jira.stsci.edu/browse/JSOCOPS-15)   ||
|                                | NRCC1C-mm TBD                                            ||
|                                | NRCC1C-nn TBD                                            |
|__Analysis Team/Roles:__        | Leads: Karl Misselt (IDT), Alicia Canipe (IST)           ||
|                                | Jarron Leisenring (Analysis/Scripts)                     ||
|                                | Thomas Beatty (TSO expertise)                            ||
|                                | Bryan Hilbert (Analysis/Scripts)                         ||
|                                | Ben Sunnquist (Analysis/Scripts)                         ||
|                                | Armin Rest (Analysis/Scripts)                            ||

## Table of Contents

* [Objective](#objective)
* [Relevant links and documentation](#links)
* [Environment for analysis](#environment)
* [Imports](#imports)
* [Data for analysis](#data)
* [Investigate the glow signature](#glow)

<a id='objective'></a>
## Objective

Since the glow accumulates much faster for subarrays (the effect is seen each time a pixel is read out), use the subarray exposures to characterize the glow that was seen in NIRISS detectors. This was noticed primarily in the long-wave data, and not so much in the shortwave data due to tuning (it is there, but we won't see it). Compare with ground test data. Note: analysis overlaps with existing dark current scripts - here we look at spatial distribution, and should see non-reference pixel edges increase more quickly. 

<a id='links'></a>
## Relevant links and documentation

|                       |                                                                                  |
|:---                   |:---                                                                              |
__JWQL dark monitor__   |                                                                             |
__NRC-09 CAR page__     |[Review Notes NRC-09](https://outerspace.stsci.edu/display/JN/Review+Notes+NRC-09)|
__NRC-09 CAP page__     |[CAP: NIRCam-09](https://outerspace.stsci.edu/display/JN/CAP%3A+NIRCam-09)        |
__Scripts__             |[NIRCam Commissioning Analysis Scripts](https://outerspace.stsci.edu/display/JN/NIRCam+Commissioning+Analysis+Scripts)                            |

<a id='environment'></a>
## Environment for analysis

Follow instructions for downloading the latest version of the pipeline to get the necessary analysis tools. Activate your environment, and then add additional tools. Note that pipeline software is not guaranteed to work on Windows machines, but it *should* work, in theory. 

[Pipeline installation](https://github.com/spacetelescope/jwst#installing-latest-releases)

```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst
```

or for a specific version:
```python
conda create -n <env_name> python
conda activate <env_name>
pip install jwst==1.2.3
```

and add additional tools used:
```python
pip install ipython jupyter matplotlib pylint pandas
```

<a id='imports'></a>
## Imports

In [ ]:
import argparse
import requests
import yaml
from glob import glob
from jwst import datamodels
import numpy as np
import pandas as pd
from astropy.io import fits, ascii
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import FuncFormatter, MaxNLocator
%matplotlib inline

In [ ]:
def side_by_side(data1, data2, vmin, vmax, title1=None, title2=None, title=None):
    """Hilbert's function to show two images side by side for easy comparison. Optionally highlight
    a given pixel with a red dot.
    
    Parameters
    ----------
    data1 : numpy.ndarray
        First image to be displayed
        
    data2 : numpy.ndarray
        Second image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
            
    title1 : str
        Title to use for first (left) plot
        
    title2 : str
        Title to use for the second (right) plot

    title : str
        String to use for the plot title
    """

    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 10))
    im = axes[0].imshow(data1, cmap=plt.cm.gray, origin='lower', vmin=vmin, vmax=vmax)
    im = axes[1].imshow(data2, cmap=plt.cm.gray, origin='lower', vmin=vmin, vmax=vmax)
    
    axes[0].set_xlabel('Pixel column')
    axes[0].set_ylabel('Pixel row')
    axes[1].set_xlabel('Pixel column')
    
    if title1:
        axes[0].set_title(title1)
    if title2:
        axes[1].set_title(title2)
        
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax, label='DN')
    if title:
        fig.suptitle(title)    

<a id='data'></a>
## Data for analysis

Data will be downloaded from MAST using the code written by Mario Gennaro and Armin Rest, and stored in our NIRCam data location: **TBD**

In [ ]:
# base_dir = '/ifs/jwst/wit/nircam/'
base_dir = '/ifs/jwst/wit/witserv/data7/nrc/'
analysis_dir = './'
ground = 'otis_long_darks'
flight = 'TBD'

In [ ]:
files = glob(base_dir+ground+'/NRC*_484_*uncal.fits')
fits.getheader(files[0],0)

<a id='glow'></a>
## Investigate the glow signature

To do: Add Ben's improved scripts. 

Generate plots of the subarrays to take a look at the behavior of the non-reference pixels near the edge. 

In [ ]:
for file in files:
    
    hdr = fits.getheader(file,0) 
    dat = fits.getdata(file,1)
    dq = fits.getdata(file,2)
    
    imgmean = np.nanmean(dat[0,0,:,:])
    imgstd = np.nanstd(dat[0,0,:,:])
    
    vmin=imgmean-2*imgstd
    vmax=imgmean+2*imgstd
    
    side_by_side(dat[0,0,:,:],dat[0,-1,:,:],vmin,vmax,hdr['FILENAME']+'\n\nFirst Group','Last Group')
    
    side_by_side(dat[0,0,hdr['SUBSTRT1']:hdr['SUBSTRT1']+20,hdr['SUBSTRT2']:hdr['SUBSTRT2']+20],
                 dat[0,-1,hdr['SUBSTRT1']:hdr['SUBSTRT1']+20,hdr['SUBSTRT2']:hdr['SUBSTRT2']+20],
                 vmin,vmax,'First Group','Last Group')
    
    side_by_side(dat[0,0,hdr['SUBSIZE1']-20:hdr['SUBSIZE1'],hdr['SUBSIZE2']-20:hdr['SUBSIZE2']],
                 dat[0,-1,hdr['SUBSIZE1']-20:hdr['SUBSIZE1'],hdr['SUBSIZE2']-20:hdr['SUBSIZE2']],
                 vmin,vmax,'First Group','Last Group')